<a href="https://colab.research.google.com/github/pipeberrio/IA-UdeA/blob/main/Copia_de_Tarea1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="banner" height="252px" width="1080px" src="https://docs.google.com/uc?export=download&id=1SqbMIjMfsMlSukiTyfMJ0VMuDlP2hGhx"  align="center" hspace="10px" vspace="0px" ></p>

# <font color='056938'> **Descripción del problema** </font>

La agencia de coperación europea esta diseñando un programa de intervención en los municipios de Colombia con el fin de favorecer la conectividad de aquellos territorios más apartados. Para el diseño de la intervensión se requiere construir una base de datos con la siguiente información de los diferentes municipios de Colombia


* Nombre del municipio
* Código DANE del municipio
* Departamento al que pertenece
* Código del Departamento
* Acto jurídico de creación del Municipio
* Área del municipio
* Localización de la cabecera municipal (latitud, longitud)
* Distancia a la cabecera municipal de la capital del departamento
* Número de habitantes del Municipio
* Nombre del alcalde
* Dirección de la alcaldía
* correo electronico de la alcaldía
* Página web de la alcaldía







Desafortudamente la información se encunetra dispersa en diferentes archivos y diferentes formatos. De momento se dispone de tres archivos:
* La división política por municipio obtenida del DANE (`DIVIPOLA_Municipios.xlsx`)
* El directorio de municipios (`Directorio-Alcaldes-2024-2027.xlsx`)
* Un archivo georeferenciado de los municipios (`co_2018_MGN_MPIO_POLITICO.geojson`)

El siguiente código carga lso archivos mencionados a este notebook


In [ ]:
!gdown 1CMEqYcmeGh0zf9rXywAFegZ1xA9IVZAb # DIVIPOLA_Municipios.xlsx
!gdown 1sqn2GDhl7ItHy-NfdMwjXlJqn0fa7wp8 # Directorio-Alcaldes-2024-2027.xlsx
!gdown 1yyOPsDXgSMJcISDH-_7HqNvqoynnOcTQ # co_2018_MGN_MPIO_POLITICO.geojson

Note que particularmente el último archivo (`co_2018_MGN_MPIO_POLITICO.geojson`) es del tipo geojson. El formato GeoJSON es un estándar de codificación para representar datos geoespaciales utilizando el formato de notación de objetos de JavaScript (JSON). El siguiente código permite visualizar dicha información georeferenciada.

In [ ]:
import plotly.express as px
import geopandas as gpd

# Load the GeoJSON file into a GeoDataFrame
gdf = gpd.read_file('co_2018_MGN_MPIO_POLITICO.geojson')

# Convert the GeoDataFrame to GeoJSON format
geojson = gdf.__geo_interface__

# Create a Plotly map
fig = px.choropleth_mapbox(gdf,
                           geojson=geojson,
                           locations=gdf.index,
                           color='MPIO_NAREA',  # Change to the column you want to visualize
                           center={"lat": 7.111788462, "lon": -75.56223474},
                           mapbox_style="carto-positron",
                           color_continuous_scale='RdYlGn',
                           zoom=4)

# Hide the color scale
fig.update_layout(coloraxis_showscale=False)

# Show the map
fig.show()



Sin embargo, note que dicho archivo contiene no solo información georeferenciada sino otra información de interes para cada municipio

![](https://docs.google.com/uc?export=download&id=1qNUSHFUGP_L0jfke4r8aRZJU4362XY6X)



## <font color='8EC044'> **Ejercicio** </font>

Construya un archivos de datos que incluya toda la información requerida. Tenga presente que de momento no es necesario que valide o corrija la información fatante o defectuosa.

Usted debera adicionar en este notebook todos los pasos seguidos para construir el archivo

**Raul Felipe Berrio Gomez 1035416370**

In [ ]:
import pandas as pd
import geopandas as gpd

# 1. Cargar los archivos
# Cargamos los archivos necesarios: la base de datos DIVIPOLA, el archivo de alcaldes y el archivo GeoJSON con los polígonos de los municipios.
divipola_df = pd.read_excel('DIVIPOLA_Municipios.xlsx', header=None)
alcaldes_df = pd.read_excel('Directorio-Alcaldes-2024-2027.xlsx')
gdf = gpd.read_file('co_2018_MGN_MPIO_POLITICO.geojson')

# 2. Renombrar columnas para facilitar la unión
# Renombramos las columnas de los archivos cargados para que tengan nombres más descriptivos y uniformes, lo que facilita la posterior unión de los DataFrames.
divipola_df.columns = ['Código Departamento', 'Nombre Departamento', 'Código Municipio', 'Nombre Municipio', 'Código Dane', 'Nombre Alcalde', 'Correo Electrónico', 'Dirección Física']
alcaldes_df.columns = ['No', 'Departamento', 'Municipio', 'Nombre Alcalde', 'Correo Electrónico', 'Dirección Física', 'Página Web', 'Unnamed']

# 3. Limpiar los datos: eliminar espacios en blanco y asegurar mayúsculas/minúsculas consistentes
# Este paso es crucial porque los problemas de coincidencia de claves en las uniones suelen deberse a diferencias en el formato del texto
# (por ejemplo, espacios en blanco adicionales o inconsistencias en mayúsculas/minúsculas).
divipola_df['Código Dane'] = divipola_df['Código Dane'].astype(str).str.strip()  # Limpieza de códigos
alcaldes_df['Municipio'] = alcaldes_df['Municipio'].astype(str).str.strip()  # Limpieza de nombres de municipios
alcaldes_df['Departamento'] = alcaldes_df['Departamento'].astype(str).str.strip()  # Limpieza de departamentos
gdf['MPIO_CCDGO'] = gdf['MPIO_CCDGO'].astype(str).str.strip()  # Limpieza de códigos en el GeoJSON

# 4. Primera unión: GeoJSON con la base DIVIPOLA
# Unimos la información geográfica de los municipios (gdf) con los datos de DIVIPOLA (divipola_df) usando el código DANE como clave.
# El parámetro 'how' se establece en 'left' para asegurarse de que todos los registros de 'gdf' permanezcan, aunque no tengan coincidencias en 'divipola_df'.
merged_df = gdf.merge(divipola_df, left_on='MPIO_CCDGO', right_on='Código Dane', how='left')

# 5. Verificación de las primeras filas después de la primera unión
# Imprimimos algunas filas para asegurarnos de que la unión se realizó correctamente y que los datos de DIVIPOLA están presentes en 'merged_df'.
print("Después de la primera unión:")
print(merged_df.head())

# 6. Segunda unión: agregar información de los alcaldes
# Unimos el DataFrame resultante ('merged_df') con la información de los alcaldes ('alcaldes_df').
# La unión se hace utilizando las claves 'MPIO_CCDGO' (código del municipio) y 'DPTO_CNMBR' (nombre del departamento) desde el lado izquierdo,
# y 'Municipio' y 'Departamento' desde el lado derecho.
# También usamos 'how' como 'left' para mantener los datos originales aunque no haya coincidencias en los alcaldes.
final_df = merged_df.merge(alcaldes_df, left_on=['MPIO_CCDGO', 'DPTO_CNMBR'], right_on=['Municipio', 'Departamento'], how='left')

# 7. Verificación de las primeras filas después de la segunda unión
# Verificamos nuevamente si la unión fue exitosa, esta vez con los datos de los alcaldes.
print("Después de la segunda unión:")
print(final_df.head())

# 8. Agrupación con 'groupby' para visualizar agrupaciones
# Agrupamos los datos por departamento (DPTO_CNMBR) para ver si hay datos disponibles por departamento y cuántos registros hay por cada uno.
grouped_df = final_df.groupby('DPTO_CNMBR').size()


# 9. Mostrar la agrupación
# Imprimimos la cantidad de registros por cada departamento para verificar si los datos están bien distribuidos.
print("Agrupación por departamento:")
print(grouped_df)

# 10. Guardar el archivo resultante en formato Excel

final_df.to_excel('datos_municipios_colombia.xlsx', index=False)

# 11. Descargar el archivo generado

from google.colab import files
files.download('datos_municipios_colombia.xlsx')
